In [1]:
import os, sys
import importlib
import numpy as np
sys.path.append("..")

In [2]:
import torch.nn
from experimenter.MultiTask.modeling import Seq2Seq, SeqDecoder

In [30]:
# Does adding parameter inside a module list attribute register it in parameters()? 

In [3]:
conf = {"device": "cpu", "out_path": ".", "model_path": ".",
        "model":{"params": {
        "dropout": 0,
        "embedding_dim": 11,
        "hidden_dim": 40,
        "inp_seq_len": [10],
        "out_seq_len": [10],
        "vocab_size": 23}}}

conf['processor'] = {}
conf['processor']['params'] = {}
conf['processor']['params']['batch_size'] = 4
conf['processor']['params']["label_encoder"] = ["text", "class"]
conf['processor']['params']['num_classes'] = [23, 3]

model = Seq2Seq(conf)

for p in model.parameters():
    print(p.type, p.shape, p.name)

<built-in method type of Parameter object at 0x1276e6d20> torch.Size([23, 11]) None
<built-in method type of Parameter object at 0x1276e6e60> torch.Size([160, 11]) None
<built-in method type of Parameter object at 0x1276e6eb0> torch.Size([160, 40]) None
<built-in method type of Parameter object at 0x1276e6f00> torch.Size([160]) None
<built-in method type of Parameter object at 0x1276e6f50> torch.Size([160]) None


In [5]:
# Define a module
decoder = SeqDecoder(conf, 
                     emb_layer=model.emb, 
                     inp_dim=conf['model']['params']['embedding_dim'], 
                     hidden_dim=40, 
                     num_classes=23,
                    max_seq_len=10)


In [6]:
# check its parameters:
for p in decoder.parameters():
    print(p.type, p.shape, p.name)

<built-in method type of Parameter object at 0x1276e6d20> torch.Size([23, 11]) None
<built-in method type of Parameter object at 0x1276f2640> torch.Size([160, 11]) None
<built-in method type of Parameter object at 0x1276f28c0> torch.Size([160, 40]) None
<built-in method type of Parameter object at 0x1276f2910> torch.Size([160]) None
<built-in method type of Parameter object at 0x1276f2820> torch.Size([160]) None
<built-in method type of Parameter object at 0x1276f2690> torch.Size([23, 40]) None
<built-in method type of Parameter object at 0x1276f2780> torch.Size([23]) None


In [42]:
# Add parameter inside a list
decoder.p = []
decoder.p.append(torch.nn.Linear(200,300))

In [10]:
# Check again to see if added parameter is showing up
# It's not! 
for p in decoder.parameters():
    print(p.type(), p.shape, p.name)

torch.FloatTensor torch.Size([23, 11]) None
torch.FloatTensor torch.Size([160, 11]) None
torch.FloatTensor torch.Size([160, 40]) None
torch.FloatTensor torch.Size([160]) None
torch.FloatTensor torch.Size([160]) None
torch.FloatTensor torch.Size([23, 40]) None
torch.FloatTensor torch.Size([23]) None
torch.FloatTensor torch.Size([300, 200]) None
torch.FloatTensor torch.Size([300]) None


In [8]:
# Add it as attribute:
decoder.p = torch.nn.Linear(200,300)

In [9]:
# Check again to see if added parameter is showing up
# It is added!  
for p in decoder.parameters():
    print(p.type, p.shape, p.name)

<built-in method type of Parameter object at 0x1276e6d20> torch.Size([23, 11]) None
<built-in method type of Parameter object at 0x1276f2640> torch.Size([160, 11]) None
<built-in method type of Parameter object at 0x1276f28c0> torch.Size([160, 40]) None
<built-in method type of Parameter object at 0x1276f2910> torch.Size([160]) None
<built-in method type of Parameter object at 0x1276f2820> torch.Size([160]) None
<built-in method type of Parameter object at 0x1276f2690> torch.Size([23, 40]) None
<built-in method type of Parameter object at 0x1276f2780> torch.Size([23]) None
<built-in method type of Parameter object at 0x1276f3e10> torch.Size([300, 200]) None
<built-in method type of Parameter object at 0x1276f3e60> torch.Size([300]) None


In [13]:
# Try to add it using ModuleList (proper way to add list of layers with registration)

# Define a module
decoder = SeqDecoder(conf, 
                     emb_layer=model.emb, 
                     inp_dim=conf['model']['params']['embedding_dim'], 
                     hidden_dim=40, 
                     num_classes=23,
                    max_seq_len=10)

decoder.p = torch.nn.ModuleList()
for i in range(10):
    decoder.p.append(torch.nn.Linear(200,300))

# Check again to see if added parameter is showing up
# It is added!  
for p in decoder.parameters():
    print(p.type, p.shape, p.name)

<built-in method type of Parameter object at 0x1276e6d20> torch.Size([23, 11]) None
<built-in method type of Parameter object at 0x127712f50> torch.Size([160, 11]) None
<built-in method type of Parameter object at 0x127712fa0> torch.Size([160, 40]) None
<built-in method type of Parameter object at 0x127705820> torch.Size([160]) None
<built-in method type of Parameter object at 0x1277055a0> torch.Size([160]) None
<built-in method type of Parameter object at 0x127705780> torch.Size([23, 40]) None
<built-in method type of Parameter object at 0x1277058c0> torch.Size([23]) None
<built-in method type of Parameter object at 0x127712d20> torch.Size([300, 200]) None
<built-in method type of Parameter object at 0x127712c80> torch.Size([300]) None
<built-in method type of Parameter object at 0x127712c30> torch.Size([300, 200]) None
<built-in method type of Parameter object at 0x127712be0> torch.Size([300]) None
<built-in method type of Parameter object at 0x127712b90> torch.Size([300, 200]) None


In [19]:
# To access parameters using their names:

for name, p in decoder.named_parameters():
    print(f"Name={name:10} \t| Type={p.type()} | Shape={p.shape}")

Name=emb.weight 	| Type=torch.FloatTensor | Shape=torch.Size([23, 11])
Name=lstm.weight_ih_l0 	| Type=torch.FloatTensor | Shape=torch.Size([160, 11])
Name=lstm.weight_hh_l0 	| Type=torch.FloatTensor | Shape=torch.Size([160, 40])
Name=lstm.bias_ih_l0 	| Type=torch.FloatTensor | Shape=torch.Size([160])
Name=lstm.bias_hh_l0 	| Type=torch.FloatTensor | Shape=torch.Size([160])
Name=linear.weight 	| Type=torch.FloatTensor | Shape=torch.Size([23, 40])
Name=linear.bias 	| Type=torch.FloatTensor | Shape=torch.Size([23])
Name=p.0.weight 	| Type=torch.FloatTensor | Shape=torch.Size([300, 200])
Name=p.0.bias   	| Type=torch.FloatTensor | Shape=torch.Size([300])
Name=p.1.weight 	| Type=torch.FloatTensor | Shape=torch.Size([300, 200])
Name=p.1.bias   	| Type=torch.FloatTensor | Shape=torch.Size([300])
Name=p.2.weight 	| Type=torch.FloatTensor | Shape=torch.Size([300, 200])
Name=p.2.bias   	| Type=torch.FloatTensor | Shape=torch.Size([300])
Name=p.3.weight 	| Type=torch.FloatTensor | Shape=torch.Size